# Pulse Sequence Design Using Reinforcement Learning

{{ explanation for my approach, what I'm trying to do }}

Currently looking to implement DDPG for RL algorithm

In [ ]:
import spinSimulation as ss
import numpy as np
import importlib
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers

In [ ]:
importlib.reload(ss)

Build the neural networks...

Using the [OpenAI SpinningUp resource](https://spinningup.openai.com/en/latest/algorithms/ddpg.html#pseudocode) for the theoretical background on DDPG, and lots of TensorFlow documentation for how to write the algorithm below.

For the policy function, I need to perform gradient ascent with the following gradient
$$
\nabla_\theta 1/|B| \sum_{s \in B} Q_\phi (s, \pi_\theta(s))
$$

And for the Q-function, perform gradient descent with
$$
\nabla_\phi 1/|B| \sum_{(s,a,r,s',d) \in B} (Q_\phi(s,a) - y(r,s',d))^2
$$

In [ ]:
# policy network
policy = keras.Sequential()
policy.add(layers.LSTM(64, input_shape = (3,)))
policy.add(layers.Dense(64))
policy.add(layers.Dense(1))

# TODO compute gradients

policy.compile(optimizer=keras.optimizers.Adam(0.01),
              loss=, 
              metrics=['accuracy'])

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.plot(range(10), np.random.normal(size=(10)), label='0')
plt.title('Random numbers')
plt.xlabel('Cycle number')
plt.ylabel('Net magnetization, real')
plt.legend()